# Inference of parameters of microtubule catastrophe 
In this problem, we use data from [Gardner, Zanic, et al., Depolymerizing kinesins Kip3 and MCAK shape cellular microtubule architecture by differential control of catastrophe, *Cell*, **147**, 1092-1103, 2011](https://doi.org/10.1016/j.cell.2011.10.037). The authors investigated the dynamics of microtubule catastrophe, the switching of a microtubule from a growing to a shrinking state.  In particular, they were interested in the time between the start of growth of a microtubule and the catastrophe event. They monitored microtubules in a single-molecule [TIRF assay](https://en.wikipedia.org/wiki/Total_internal_reflection_fluorescence_microscope) by using tubulin (the monomer that comprises a microtubule) that was labeled with a fluorescent marker. As a control to make sure that fluorescent labels and exposure to laser light did not affect the microtubule dynamics, they performed a similar experiment using differential interference contrast (DIC) microscopy. They measured the time until catastrophe with labeled and unlabeled tubulin. We will carefully analyze the data and make some conclusions about the processes underlying microtubule catastrophe.

In the file `gardner_mt_catastrophe_only_tubulin.csv` (which you can download [here](https://s3.amazonaws.com/bebi103.caltech.edu/data/gardner_mt_catastrophe_only_tubulin.csv)), we have observed catastrophe times of microtubules with different concentrations of tubulin. We will consider the experiment run with a tubulin concentration of 12 µM. So, our data set consists of a set of measurements of the amount of time to catastrophe. We will consider four models for microtubule catastrophe.

- Model 1: The time to catastrophe is Exponentially distributed.
- Model 2: The time to catastrophe is Gamma distributed.
- Model 3: The time to catastrophe is Weibull distributed.
- Model 4: Catastrophe happens upon the arrival of the second of two Poisson processes, each of which arrive at different rates.

Note that these descriptions are for the likelihood; we have not specified priors.

**a)**  Describe the first three models as stories. Give physical descriptions of the meanings of their parameters. Describe how these models are related to each other. Recall that we worked out the likelihood for model 4 in a previous exercise. How is it related to the other three models?

**b)** Build full Bayesian generative models for each of the models. Be sure to perform prior predictive checks. After building the model, perform parameter estimates, and then do posterior predictive checks.

**c)** Based on the graphical analysis, do you prefer one model versus another? We will be discussing quantitative model comparison in coming weeks to enable to more detailed analysis.

_____________
# Answers

In [37]:
import pandas as pd
import numpy as np
rng = np.random.default_rng()
import cmdstanpy
import arviz as az
import iqplot
import bebi103
import bokeh.io
import os
from bokeh.layouts import row as bokeh_row
from bokeh.layouts import column as bokeh_column
bokeh.io.output_notebook()


Loading BokehJS ...

In [38]:
data_path = '../data/'
df = pd.read_csv(data_path + 'gardner_mt_catastrophe_only_tubulin.csv', comment='#')
evidence = df.loc[:,'12 uM'].values
data = {'N': evidence.shape[0], 'y': evidence}
print(data['N'])


692


## a.
<!--a. Describe the first three models (Exponential, Gamma, Weibull) as stories. 
Give physical descriptions of the meanings of their parameters. 
Describe how these models are related to each other. 
Recall that we worked out the likelihood for model 4 in a previous exercise. 
How is it related to the other three models?

They were interested in the time between the start of growth of a microtubule and the catastrophe event. 
They monitored microtubules in a single-molecule TIRF assay by using a fluorescent tubulin.
They measured the time until catastrophe with labeled and unlabeled tubulin.
we have observed catastrophe times of microtubules with different concentrations of tubulin. We will consider the experiment run with a tubulin concentration of 12 µM. So, our data set consists of a set of measurements of the amount of time to catastrophe.

 -->
### Exponential distribution
- **Story**: The inter-arrival time of a Poisson process is Exponentially distributed. <br>
- **Relevance**: We treat catastrophe events as Poisson proccesses and hence the time intervals from grpwth to catastrophe might be exponentially distributed.<br>
- - **Parameters**: $\beta$ is the rate of events.

### Gamma distribution
- **Story**: The amount of time we have to wait for $\alpha$ arrivals of a Poisson process is Gamma distributed. If events $X_1,X_2,...,X_\alpha$ are exponentially distributed, then $X_1+X_2+...+X_\alpha$ is Gamma distributed. <br>
- **Relevance**: I already explained how the Exponential is relevant and since the Exponential distribution is a special case of the Gamma distribution with parameter $\alpha=1$, it is easy to see how the Gamma distribution can also be applied here. We can use it if we look at the combined measurements of catastrophe events.
- - **Parameters**: $\beta$ is the rate of events and $\alpha$ is the number of catastrophe events.

Correction: We assume hidden events/steps that are actually part of the process. Therefor each measurement actually encompasses a number of hidden events.
<!-- TODO: FIX -->

### Weibull distribution
- **Story**: Similary to the Exponential distribution, it describes waiting times for arrival of a process. For $\alpha>1$ the longer we have waited, the more likely the event is to arrive, and vice versa for $\alpha<1$. If $y$ is Exponentially distributed, $y^\alpha$ is Weibull distributed.
- **Relevance**: If we assume that longer constructions are more fragile, then $\alpha$ will be larger than 1. Meaning that as time of elongation increases, the probability of catastrophe increase. The Exponential distribution is a special case of the Weibull distribution with parameter $\alpha=1$.
- - **Parameters**: $\alpha$ dictates the shape of the curve - As it increases the curve shifts to the right (likelihood of event increases as time progresses). $\sigma$ is a scale parameter that dictates the rate of catastrophe events arrivals.

### Relation to a "bi-Poisson" model
<!-- TODO: Talk about how the 4th model is related to the others -->
- **Exponential:** We can treat such occurence as two exponentially distributed events followed by one-another. Hence this is the sum of two Exponential distributions.
- **Gamma:** According to this model, catastrophe happens upon the arrival of two sequential Poisson processes, so if the rate of these two events is equal, we can say that this is Gamma with $\alpha=2$.
- **Weibull:** In the most simple manner, the model can be described as the sum of two independent Weibull distributions (just like it can be for Exponential). But we can imagine a more complex scenario where the waiting time of the 2nd process includes the time it took for the arrival of the 1st process, meaning that when the 1st arrival takes longer, the 2nd arrival is more likely to be shorter.



## b.
<!--
Build full Bayesian generative models for each of the models. 
Be sure to perform prior predictive checks. After building the model, perform parameter estimates, and then do posterior predictive checks. 
-->
To be unbiased, I'm going to estimate that the time intervals are in the order of magnitude of the range $10^{-2}$ seconds and $10^3$ seconds (*bet the farm*) and I'm adjusting my prior predictive checks according to that.

All of the posterior predictive checks are plotted together towards the end.

### Model 1: Exponential

In [39]:
## Prior predictive checks
beta_ = 10**rng.normal(-0.5, 1.276, size=1_000)  # rate parameter
syn_events = np.array([rng.exponential(1/b, size=1000) for b in beta_])
bokeh.io.show(
    bebi103.viz.predictive_ecdf(
        syn_events, 
        x_axis_label="Time intervals (s)", 
        x_axis_type='log',
        title="Predictive Prior for Exponential Distribution"
        )
    )


In [40]:
#%% Define stan code
exponential_stan_code="""
data {
    // Number of events
    int<lower=1> N;

    // interspike intervals
    array[N] real<lower=0> y;
}

parameters {
//    real<lower=0> beta_;
    real log10_beta;


}
transformed parameters {
   real<lower=0> beta_ = 10^log10_beta;
}

model {
    // Prior
    log10_beta ~ normal(-0.5, 1.276);

    // Likelihood
    y ~ exponential(beta_);
}

generated quantities {
    // Posterior predictive samples
    array[N] real<lower=0> y_pred;

    for (n in 1:N) {
        y_pred[n] = exponential_rng(beta_);
    }

    // Log-likelihood
    array[N] real log_lik;
    for (n in 1:N) {
        log_lik[n] = exponential_lpdf(y[n] | beta_);
    }
}
"""
#%% Write stan code to file
expo_stan_file_name = 'TomerAntman-78-exponential.stan'

if not os.path.exists(expo_stan_file_name):
    with open(expo_stan_file_name, 'w') as f:
        f.write(exponential_stan_code)

#%% Compile model
sm = cmdstanpy.CmdStanModel(stan_file=expo_stan_file_name)

In [41]:
with bebi103.stan.disable_logging():

    expo_samples = sm.sample(
            data=data,
            chains=4,
            iter_sampling=1000,
        )
    expo_samples = az.from_cmdstanpy(posterior=expo_samples, posterior_predictive='y_pred')

y_pred_exp = (
    expo_samples.posterior_predictive['y_pred']
    .stack({"sample": ("chain", "draw")})
    .transpose("sample", "y_pred_dim_0")
)

chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

In [42]:
predicted_betas = expo_samples.posterior.beta_.values.flatten()
# plot predicted_betas:
p = iqplot.histogram(
    predicted_betas,
    x_axis_label='Beta (rate parameter)',
    rug_kwargs={'alpha':0.05, 'color':'dodgerblue'},
    line_kwargs={'color':'dodgerblue'},
    title="Marginal distributions of 'Exponential' parameters:"
)
bokeh.io.show(p)

### Model 2: Gamma

In [43]:
## Prior
betas = 10**rng.normal(0, 1.276, size=1_000)  # rate parameter
alphas = 10**rng.normal(0.85, 0.434, size=1_000)  # shape parameter
syn_events = np.array([rng.gamma(a, 1/b, size=1000) for a, b in zip(alphas, betas)])
bokeh.io.show(
    bebi103.viz.predictive_ecdf(
        syn_events, 
        x_axis_label="Time intervals (s)", 
        x_axis_type='log',
        title="Predictive Prior for Gamma Distribution"
        )
    )


In [44]:
#%% Define stan code
gamma_stan_code="""
data {
    // Number of events
    int<lower=1> N;

    // interspike intervals
    array[N] real<lower=0> y;
}

parameters {
//    real<lower=0> beta_;
    real log10_beta;
    real log10_alpha;

}
transformed parameters {
   real<lower=0> beta_ = 10^log10_beta;
   real<lower=1> alpha = 10^log10_alpha;
}

model {
    // Prior
    log10_beta ~ normal(0, 1.276);
    log10_alpha ~ normal(0.85, 0.434);

    // Likelihood
    y ~ gamma(alpha, beta_);
}

generated quantities {
    // Posterior predictive samples
    array[N] real<lower=0> y_pred;

    for (n in 1:N) {
        y_pred[n] = gamma_rng(alpha, beta_);
    }
    
    // Log-likelihood
    array[N] real log_lik;
    for (n in 1:N) {
        log_lik[n] = gamma_lpdf(y[n] | alpha, beta_);
    }
}
"""
#%% Write stan code to file
gamma_stan_file_name = 'TomerAntman-78-gamma.stan'

if not os.path.exists(gamma_stan_file_name):
    with open(gamma_stan_file_name, 'w') as f:
        f.write(gamma_stan_code)

#%% Compile model
sm = cmdstanpy.CmdStanModel(stan_file=gamma_stan_file_name)

In [45]:
with bebi103.stan.disable_logging():

    gamma_samples = sm.sample(
            data=data,
            chains=4,
            iter_sampling=1000,
        )
    gamma_samples = az.from_cmdstanpy(posterior=gamma_samples, posterior_predictive='y_pred')

y_pred_gamma = (
    gamma_samples.posterior_predictive['y_pred']
    .stack({"sample": ("chain", "draw")})
    .transpose("sample", "y_pred_dim_0")
)


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

In [71]:
print("Marginal distributions of 'Gamma' parameters:")
gamma_corner = bebi103.viz.corner(
    gamma_samples,
    show_contours=True, 
    parameters=['alpha', 'beta_'],
    frame_width=250,
    frame_height=250
    )
gamma_corner.children[1].children[0].children[0].renderers[0].glyph.line_color = 'dodgerblue'
gamma_corner.children[1].children[1].children[1].renderers[0].glyph.line_color = 'orange'
gamma_corner.children[1].children[1].children[0].xaxis.axis_line_color =  'dodgerblue'
gamma_corner.children[1].children[1].children[0].yaxis.axis_line_color = 'orange'
bokeh.io.show(gamma_corner)

Marginal distributions of 'Gamma' parameters:


### Model 3: Weibull

In [ ]:
## Prior
sigmas = 10**rng.normal(0.5, 1.276, size=1_000)
alphas = 10**rng.normal(0.35, 0.3316, size=1_000)  
weibull_syn_events = np.array([s*rng.weibull(a, size=1000) for a,s in zip(alphas, sigmas)])
bokeh.io.show(
    bebi103.viz.predictive_ecdf(
        weibull_syn_events, 
        x_axis_label="Time intervals (s)", 
        x_axis_type='log',
        title="Predictive Prior for Weibull Distribution"
        )
    )


In [48]:
#%% Define stan code
weibull_stan_code="""
data {
    // Number of events
    int<lower=1> N;

    // interspike intervals
    array[N] real<lower=0> y;
}

parameters {
//    real<lower=0> beta_;
    real log10_sigma;
    real log10_alpha;


}
transformed parameters {
   real<lower=0> sigma = 10^log10_sigma;
   real<lower=0> alpha = 10^log10_alpha;
}

model {
    // Prior
    log10_sigma ~ normal(0.5, 1.276);
    log10_alpha ~ normal(0.35, 0.3316);

    // Likelihood
    y ~ weibull(alpha, sigma);
}

generated quantities {
    // Posterior predictive samples
    array[N] real<lower=0> y_pred;

    for (n in 1:N) {
        y_pred[n] = weibull_rng(alpha, sigma);
    }

    // Log-likelihood
    array[N] real log_lik;
    for (n in 1:N) {
        log_lik[n] = weibull_lpdf(y[n] | alpha, sigma);
    }

}
"""
#%% Write stan code to file
weibull_stan_file_name = 'TomerAntman-78-weibull.stan'

if not os.path.exists(weibull_stan_file_name):
    with open(weibull_stan_file_name, 'w') as f:
        f.write(weibull_stan_code)

#%% Compile model
sm = cmdstanpy.CmdStanModel(stan_file=weibull_stan_file_name)

15:52:42 - cmdstanpy - INFO - compiling stan file /tmp/tmpdmltwkz4/tmpbz2ky__x.stan to exe file /home/tomerantman/projects/wis-stats/Statistical inference with Markov chain Monte Carlo/TomerAntman-78-weibull
15:53:03 - cmdstanpy - INFO - compiled model executable: /home/tomerantman/projects/wis-stats/Statistical inference with Markov chain Monte Carlo/TomerAntman-78-weibull


In [49]:
with bebi103.stan.disable_logging():
    weibull_samples = sm.sample(
            data=data,
            chains=4,
            iter_sampling=1000,
        )
    weibull_samples = az.from_cmdstanpy(posterior=weibull_samples, posterior_predictive='y_pred')

y_pred_weibull = (
    weibull_samples.posterior_predictive['y_pred']
    .stack({"sample": ("chain", "draw")})
    .transpose("sample", "y_pred_dim_0")
)


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

In [ ]:
print("Marginal distributions of 'Weibull' parameters:")
weibull_corner = bebi103.viz.corner(
    weibull_samples,
    show_contours=True, 
    parameters=['alpha', 'sigma'],
    frame_width=250,
    frame_height=250
    )
weibull_corner.children[1].children[0].children[0].renderers[0].glyph.line_color = 'dodgerblue'
weibull_corner.children[1].children[1].children[1].renderers[0].glyph.line_color = 'orange'
weibull_corner.children[1].children[1].children[0].xaxis.axis_line_color =  'dodgerblue'
weibull_corner.children[1].children[1].children[0].yaxis.axis_line_color = 'orange'

bokeh.io.show(weibull_corner)

Marginal distributions of 'Weibull' parameters:


### Model 4:

In [51]:
## Prior
beta_1 = 10**rng.normal(-0.5, 1.276, size=1_000)  
beta_2 = 10**rng.normal(-0.5, 1.276, size=1_000) 
dupexp_syn_events = np.array([rng.exponential(1/b1, size=1000) + rng.exponential(1/b2, size=1000) for b1, b2 in zip(beta_1, beta_2)])
bokeh.io.show(
    bebi103.viz.predictive_ecdf(
        dupexp_syn_events, 
        x_axis_label="Time intervals (s)", 
        x_axis_type='log',
        title="Predictive Prior for 'Hypo-Exponential' Distribution"
        )
    )

In [52]:
#%% Define stan code
hypoexponential_stan_code="""
functions {
  // Log probability density function for two-stage hypoexponential distribution
  real hypoexponential_2stage_lpdf(real t, real beta1, real beta2) {
    // Probability density is zero for negative times (time cannot be negative)
    if (t < 0) {
      return negative_infinity();
    }
    
    // Main PDF formula: f(t) = (beta1 * beta2)/(beta2 - beta1) * (exp(-beta1*t) - exp(-beta2*t))
    // Compute this in log space for numerical stability
    real log_coeff = log(beta1) + log(beta2) - log(abs(beta2 - beta1));
        
    return log_coeff + log(abs(exp(-beta1 * t) - exp(-beta2 * t)));
  }
  
  // Random number generator for two-stage hypoexponential distribution
  real hypoexponential_2stage_rng(real beta1, real beta2) {
    // Generates a sample by summing two independent exponential random variables
    // to directly models the biological process: total time = time for process 1 + time for process 2
    return exponential_rng(beta1) + exponential_rng(beta2);
  }
  
}

data {
    int<lower=1> N;

    // Each y_i value represents the time for two sequential biochemical processes
    array[N] real<lower=0> y;
}

parameters {
    real log10_beta1;  // log10 of rate parameter for first biochemical process  
    real log10_beta2;  // log10 of rate parameter for second biochemical process
}

transformed parameters {
    real<lower=0> beta1 = 10^log10_beta1;  // Rate parameter for first process
    real<lower=0> beta2 = 10^log10_beta2;  // Rate parameter for second process
}

model {
    // Prior
    log10_beta1 ~ normal(-0.5, 1.276);  // Prior for log10(beta1)
    log10_beta2 ~ normal(-0.5, 1.276);  // Prior for log10(beta2)

    // Likelihood (total time for both processes)
    for (n in 1:N) {
      // When beta1 ≈ beta2, the formula breaks because of log(0).
      // In this case, the story fits a Gamma distribution with alpha=2, beta=beta1
      if (abs(beta1 - beta2) < 1e-10) {
        y[n] ~ gamma(2, beta1);
      } else {
        // This automatically calls hypoexponential_2stage_lpdf(y[n] | beta1, beta2):
        y[n] ~ hypoexponential_2stage(beta1, beta2);
      }
    }
}

generated quantities {
    // Posterior predictive samples
    array[N] real<lower=0> y_pred;
    
    // Generate one predicted observation per each real observation
    for (n in 1:N) {
        y_pred[n] = hypoexponential_2stage_rng(beta1, beta2);
    }
    
    // Log-likelihood 
    array[N] real log_lik;
    for (n in 1:N) {
        log_lik[n] = hypoexponential_2stage_lpdf(y[n] | beta1, beta2);
    }
}
"""
#%% Write stan code to file
hypoexponential_stan_file_name = 'TomerAntman-78-hypoexponential.stan'

if not os.path.exists(hypoexponential_stan_file_name):
    with open(hypoexponential_stan_file_name, 'w') as f:
        f.write(hypoexponential_stan_code)

#%% Compile model
sm = cmdstanpy.CmdStanModel(stan_file=hypoexponential_stan_file_name)

15:53:06 - cmdstanpy - INFO - compiling stan file /tmp/tmpzh44p5lq/tmp_n3471p9.stan to exe file /home/tomerantman/projects/wis-stats/Statistical inference with Markov chain Monte Carlo/TomerAntman-78-hypoexponential
15:53:16 - cmdstanpy - INFO - compiled model executable: /home/tomerantman/projects/wis-stats/Statistical inference with Markov chain Monte Carlo/TomerAntman-78-hypoexponential


In [53]:
with bebi103.stan.disable_logging():
    hypoexponential_samples = sm.sample(
            data=data,
            chains=4,
            iter_sampling=1000,
        )
    hypoexponential_samples = az.from_cmdstanpy(posterior=hypoexponential_samples, posterior_predictive='y_pred')

y_pred_hypoexponential = (
    hypoexponential_samples.posterior_predictive['y_pred']
    .stack({"sample": ("chain", "draw")})
    .transpose("sample", "y_pred_dim_0")
)


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

In [70]:

print("Marginal distributions of 'Hypoexponential' parameters:")
hypoexponential_corner = bebi103.viz.corner(
    hypoexponential_samples,
    show_contours=True, 
    parameters=['beta1', 'beta2'],
    frame_width=250,
    frame_height=250
    )
hypoexponential_corner.children[1].children[0].children[0].renderers[0].glyph.line_color = 'dodgerblue'
hypoexponential_corner.children[1].children[1].children[1].renderers[0].glyph.line_color = 'orange'
hypoexponential_corner.children[1].children[1].children[0].xaxis.axis_line_color =  'dodgerblue'
hypoexponential_corner.children[1].children[1].children[0].yaxis.axis_line_color = 'orange'
bokeh.io.show(hypoexponential_corner)

Marginal distributions of 'Hypoexponential' parameters:


### Plot Posterior Predictive Checks:

In [55]:

p_exp = bebi103.viz.predictive_ecdf(y_pred_exp, 
                                    evidence, 
                                    x_axis_type='log', 
                                    x_axis_label="",
                                    height=350, width=350,
                                    title="Exponential model"
                                    )
p_gamma = bebi103.viz.predictive_ecdf(y_pred_gamma, 
                                      evidence, 
                                      x_axis_type='log', 
                                      x_axis_label="",
                                      y_axis_label="",
                                      height=350, width=350,
                                      title="Gamma model",
                                      )
p_weibull = bebi103.viz.predictive_ecdf(y_pred_weibull, 
                                        evidence, 
                                        x_axis_type='log', 
                                        x_axis_label="",
                                        y_axis_label="",
                                        height=350, width=350,
                                        title="Weibull model"
                                        )

p_hypoexp = bebi103.viz.predictive_ecdf(y_pred_hypoexponential,
                                        evidence, 
                                        x_axis_type='log', 
                                        x_axis_label="",
                                        y_axis_label="",
                                        height=350, width=350,
                                        title="Hypoexponential model"
                                        )

p_exp_diff = bebi103.viz.predictive_ecdf(y_pred_exp, 
                                         data=evidence, 
                                         diff='ecdf', 
                                         x_axis_type='log', 
                                         x_axis_label="Time intervals (s)",
                                         height=350, width=350
                                         )
p_gamma_diff = bebi103.viz.predictive_ecdf(y_pred_gamma, 
                                           data=evidence, 
                                           diff='ecdf', 
                                           x_axis_type='log', 
                                           x_axis_label="Time intervals (s)",
                                           y_axis_label="",
                                           height=350, width=350
                                           )
p_weibull_diff = bebi103.viz.predictive_ecdf(y_pred_weibull, 
                                             data=evidence, 
                                             diff='ecdf', 
                                             x_axis_type='log', 
                                             x_axis_label="Time intervals (s)",
                                             y_axis_label="", 
                                             height=350, width=350
                                             )
p_hypoexp_diff = bebi103.viz.predictive_ecdf(y_pred_hypoexponential, 
                                             data=evidence, 
                                             diff='ecdf', 
                                             x_axis_type='log', 
                                             x_axis_label="Time intervals (s)",
                                             y_axis_label="", 
                                             height=350, width=350
                                             )
col1 = bokeh_column(p_exp, p_exp_diff)
col1.children[0].toolbar_location = None
col1.children[1].toolbar_location = None

col2 = bokeh_column(p_gamma, p_gamma_diff)
col2.children[0].toolbar_location = None
col2.children[1].toolbar_location = None

col3 = bokeh_column(p_weibull, p_weibull_diff)
col3.children[0].toolbar_location = None
col3.children[1].toolbar_location = None

col4 = bokeh_column(p_hypoexp, p_hypoexp_diff)
bokeh.io.show(bokeh_row(col1, col2, col3, col4))

## c.
I added log-likelihood to all stan files:

In [56]:
models_dict = {
    "Exponential": expo_samples, 
    "Gamma": gamma_samples, 
    "Weibull": weibull_samples, 
    "Hypoexponential": hypoexponential_samples
}
az.compare(models_dict, ic="waic", scale="deviance")

/home/tomerantman/projects/wis-stats/.pixi/envs/default/lib/python3.13/site-packages/arviz/stats/stats.py:1667: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  warnings.warn(


,rank,elpd_waic,p_waic,elpd_diff,weight,se,dse,warning,scale
Gamma,0,9278.731703,2.321340,0.000000,0.924598,45.556230,0.000000,False,deviance
Weibull,1,9313.907600,2.878597,35.175897,0.000000,48.123430,10.084734,True,deviance
Hypoexponential,2,9326.297326,0.788897,47.565623,0.075402,36.855812,13.924439,False,deviance
Exponential,3,9608.616584,0.378783,329.884881,0.000000,31.992207,28.388313,False,deviance


#### Answer
According to WAIC, **Gamma is the best fit distribution for our data.** This is also indicated by our "ECDF" and "ECDF Difference" plots where the Gamma distribution plot is the only one that contains all of the evidence data within 2 standard deviations (orange remains in the blue).